In [1]:
import collections
import requests
import json
import re
import os
import sys

with open('../../data/speeches_20.jsonl', 'r', encoding = 'utf8') as fp:
    data = list(fp)
speeches = []
for line in data:
    speeches.append(json.loads(line))

In [2]:
# globale items
regex = "{(.*?)}" # hier die genaue Definition der Kommentare vom Parser <<< >>>
kommentare = []

In [3]:
def groupSpeechesByParty(speeches):
    groupedSpeeches = {}
    for speech in speeches:
        tpo = speech['party']
        if tpo in groupedSpeeches:
            groupedSpeeches[tpo].append(speech)
        else:
            groupedSpeeches[tpo] = [speech]
    return groupedSpeeches

In [4]:
groupbyparty = groupSpeechesByParty(speeches)
groupbyparty.keys()

dict_keys(['SPD', 'CDU/CSU', 'BÜNDNIS\xa090/DIE GRÜNEN', 'FDP', 'AfD', 'DIE LINKE', 'Fraktionslos', 'BÜNDNIS 90/DIE GRÜNEN'])

In [5]:
for speech in speeches:
    if 'BÜNDNIS\xa090/DIE GRÜNEN' in speech['party']:
        speech.update({'party': 'BÜNDNIS 90/DIE GRÜNEN'})
    #print(speech['party'])

print(len(speeches)) # Anzahl aller Reden
for ele in groupbyparty:
    print(len(groupbyparty[ele]))
# Überprüfung mit bspw.: len(groupbyparty['SPD'])

2777
698
607
354
395
353
241
26
103


In [8]:
for speech in speeches:
    if 'BÜNDNIS\xa090/DIE GRÜNEN' in speech['party']:
        speech.update({'party': 'DIE GRÜNEN'})
        print(speech['party'])